In [42]:
import pandas as pd 

train = pd.read_csv('../data/kaggle_bias/train.csv')

In [10]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install datasets transformers huggingface_hub


  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/73/75/ead8e4489217835fd8004c9c89dad8217deacd7e2591340dbe249b58c29f/datasets-2.17.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/b7/3a/74a609706ef4430fe6d041a3b8d209882c15440b695e373fe26d48c6f35c/xxhash-3.4.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/50/15/b56e50e8debaf439f44befec5b2af11db85f6e0f344c3113ae0be0593a91/multiprocess-0.70.16-py311-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db8158cfec1eee3

In [43]:
train = train.dropna(subset=['comment_text'])
train.reset_index(drop=True, inplace=True)
train['bi_target'] = (train['target'] >= 0.5).astype(int)

In [44]:
train_sample = train.sample(frac=0.2)

In [45]:
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['comment_text']
        label = self.dataframe.iloc[idx]['bi_target']
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=215, return_tensors="pt")
        
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        
        return input_ids, attention_mask, torch.tensor(label, dtype=torch.long)

In [46]:
from transformers import DistilBertTokenizer
from sklearn.model_selection import train_test_split

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_df, val_df = train_test_split(train_sample, test_size=0.1)

train_dataset = MyDataset(train_df, tokenizer)
val_dataset = MyDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [47]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# makes the model run on the GPU instead of CPU
model.cuda()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [48]:
from transformers import AdamW
from tqdm.auto import tqdm

# AdamW optimizer is apparently really good for DistilBERT?  Will write more in docs
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):  
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for input_ids, attention_mask, labels in progress_bar:
        # Move the training to the GPU
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()
        labels = labels.cuda()

        # Set gradients to zero for training
        model.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        progress_bar.set_postfix({'loss': loss.item()})

C:\Users\matth\anaconda3\envs\torchpower\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1:   0%|          | 0/20305 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/20305 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/20305 [00:00<?, ?it/s]

In [49]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


model.eval()  # Transformers built in model evaluation kit 
true_labels = []
pred_labels = []

with torch.no_grad():  # Disable gradient calculation
    for input_ids, attention_mask, labels in val_loader:
        # Move tensors to the GPU
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()
        labels = labels.cuda()
        
        # Forward pass, get logit predictions
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Move logits and labels to CPU
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        
        # Store predictions and true labels
        true_labels.append(label_ids)
        pred_labels.append(logits.argmax(axis=1))

# Calculate our accuracy metrics
true_labels = np.concatenate(true_labels)
pred_labels = np.concatenate(pred_labels)


accuracy = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.9418250318577206
Precision: 0.650619985412108
Recall: 0.6097060833902939
F1 Score: 0.629498941425547
